In [2]:
import findspark
findspark.init('/home/ubuntu/spark-3.2.0-bin-hadoop3.2')
DATA_PATH = '../data/Spark_for_Machine_Learning/Spark Streaming/'

In [3]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import explode
from pyspark.sql.functions import split

In [4]:
spark = SparkSession.builder.appName('StructStream').getOrCreate()

Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
21/12/31 16:32:00 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [5]:
## Example from docs
lines = spark \
    .readStream \
    .format("socket") \
    .option("host", "127.0.0.1") \
    .option("port", 5555) \
    .load()

# Split the lines into words
words = lines.select(
   explode(
       split(lines.value, " ")
   ).alias("word")
)

# Generate running word count
wordCounts = words.groupBy("word").count()

21/12/31 16:32:05 WARN TextSocketSourceProvider: The socket source should not be used for production applications! It does not support recovery.


In [6]:
query = wordCounts \
    .writeStream \
    .outputMode("complete") \
    .format("console") \
    .start()


# Intended to listen to TweetRead.py
query.awaitTermination()

21/12/31 16:32:08 WARN ResolveWriteToStream: Temporary checkpoint location created which is deleted normally when the query didn't fail: /tmp/temporary-d5901068-6140-4186-bdc1-8f5416a26af1. If it's required to delete it under any circumstances, please set spark.sql.streaming.forceDeleteTempCheckpointLocation to true. Important to know deleting temp checkpoint folder is best effort.
21/12/31 16:32:08 WARN ResolveWriteToStream: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.


-------------------------------------------
Batch: 0
-------------------------------------------
+----+-----+
|word|count|
+----+-----+
+----+-----+



-------------------------------------------
Batch: 1
-------------------------------------------
+--------------------+-----+
|                word|count|
+--------------------+-----+
|             UConn’s|    1|
|                  If|    1|
|               gotta|    1|
|               field|    2|
|                 him|    1|
|      the…@underlouv|    1|
|                 who|    1|
|                 not|    1|
|https://t.co/6zfl...|    1|
|                  ur|    1|
|             because|    1|
|               izuku|    1|
|          @akuisgay:|    1|
|                 for|    2|
|               Lasso|    1|
|                 how|    1|
|              Sports|    1|
|                 top|    1|
|                they|    1|
|                with|    1|
+--------------------+-----+
only showing top 20 rows



-------------------------------------------
Batch: 2
-------------------------------------------
+--------------------+-----+
|                word|count|
+--------------------+-----+
|             UConn’s|    1|
|            positive|    1|
|            spending|    1|
|                  If|    1|
|               gotta|    1|
|               field|    2|
|                 him|    1|
|                  🏀|    1|
|      the…@underlouv|    1|
|                 who|    1|
|https://t.co/6zfl...|    1|
|                 not|    1|
|                  ur|    1|
|             because|    1|
|               izuku|    1|
|          @akuisgay:|    1|
|                 for|    3|
|               Lasso|    1|
|                 how|    1|
|              Sports|    1|
+--------------------+-----+
only showing top 20 rows



-------------------------------------------
Batch: 3
-------------------------------------------
+--------------------+-----+
|                word|count|
+--------------------+-----+
|             UConn’s|    1|
|            positive|    1|
|                  Em|    1|
|            spending|    1|
|                  If|    1|
|               gotta|    1|
|                  11|    1|
|               field|    2|
|                 him|    1|
|                  🏈|    1|
|             Daddy’s|    1|
|                  🏀|    1|
|      the…@underlouv|    1|
|                 a…7|    1|
|         documentary|    1|
|                  em|    1|
|                 who|    1|
|https://t.co/6zfl...|    1|
|                 not|    1|
|                 boy|    1|
+--------------------+-----+
only showing top 20 rows



21/12/31 16:33:06 WARN TextSocketMicroBatchStream: Stream closed by 127.0.0.1:5555


-------------------------------------------
Batch: 4
-------------------------------------------
+--------------+-----+
|          word|count|
+--------------+-----+
|   interaction|    1|
|       UConn’s|    1|
|      positive|    1|
|      spending|    1|
|            Em|    1|
|            If|    1|
|        color:|    1|
|         gotta|    1|
|            11|    1|
|           him|    1|
|         could|    1|
|         field|    2|
|            🏈|    1|
|            🏀|    1|
|       Daddy’s|    1|
|           a…7|    1|
|the…@underlouv|    1|
|   documentary|    1|
|            em|    1|
|           who|    1|
+--------------+-----+
only showing top 20 rows



KeyboardInterrupt: 